Системы аналитических вычислений.

Студент: Пермяков Никита Александрович, М8О-208Б-19

Лабораторная работа № 2

### Задание 1. Вариант А. дихотомия и метод Ньютона

Поиск решения осуществляется путём построения последовательных приближений и основан на принципах простой итерации. Метод обладает квадратичной сходимостью. 

In [1]:
# Дано
f(x) = x^3 - 9*x^2 + 23*x - 15
a = (0, 2, 0.5)
b = (0, 2, 0.5)
step = (1, 2, 1)

xmin = -1
xmax = 5.5
ymin = -4
ymax = 5

eps = 1e-5

In [2]:
# Решим уравнение
solve([f(x) == 0], x)

[x == 1, x == 5, x == 3]

In [3]:
# Метод дихотомии - ищем точку пересечения с Ox
def dichotomy(fun, a, b, eps=1e-5):
    if fun(x=a) * fun(x=b) > 0:
        return None
    m = (a + b) / 2.
    if abs(fun(x=m).n()) < eps:
        return m
    if fun(x=m) * fun(x=a) < 0:
        return dichotomy(fun, a, m)
    else:
        return dichotomy(fun, m, b)

In [4]:
# Метод Ньютона - ищем точку пересечения с Ox
def newton(fun, a, b, eps=1e-5):
    x1 = a
    x2 = (a + b) / 2
    df = fun.derivative()
    while abs(x2 - x1) >= eps:
        x1 = x2
        x2 = x1 - fun(x=x1) / df(x=x1)
    if abs(fun(x=x1)) <= 1e-3:
        return x1
    return None

In [5]:
def setup_graph(func):
    return point((float(func), 0), color='green', aspect_ratio=1, size=30)

@interact
def func_interact(a=a, b=b, step=step):    
    # f, eps - global
    
    # дихотомия
    di = [dichotomy(f, a, b, eps)]
    # Добавляем по шагу
    di.append(dichotomy(f, a + step, b + step, eps))
    di.append(dichotomy(f, a + 2 * step, b + 2 * step, eps))
    
    # Ньютон
    newt = [newton(f, a, b, eps)]
    # Добавляем по шагу
    newt.append(newton(f, a + step, b + step, eps))
    newt.append(newton(f, a + 2 * step, b + 2 * step, eps))
    
    
    # устанавливаем параметры 
    graph = [plot(f, (x, xmin, xmax), ymin=ymin, ymax=ymax, aspect_ratio=1, figsize=[3,2], color='black')] * 6
    
    for i in range(len(graph)):
        if i <= 2 and di[i] is not None:
            graph[i] += setup_graph(di[i])
        if i > 2 and newt[i-3] is not None:
            graph[i] += setup_graph(newt[i-3])

    
    str_ = "Корни, полученные методом дихотомии: "
    for d in di:
        if d is not None:
            str_ += str(round(float(d))) + " "
            
    str_ += "\nКорни, полученные методом Ньютона: "
    for n in newt:
        if n is not None:
            str_ += str(round(float(n))) + " "
            
            
    # рисуем
    show("метод дихотомии")        
    graphics_array([[graph[0], graph[1], graph[2]]]).show(frame=True)
    show("метод Ньютона")
    graphics_array([[graph[3], graph[4], graph[5]]]).show(frame=True)            
    show(str_)

Interactive function <function func_interact at 0x7fb88584f0d0> with 3 widgets
  a: FloatSlider(value=1.0, min…

 результат, полученный методом дихотомии и Ньютона совпадает с результатом solve для каждого корня

### Задание 2. Вариант 2. полиномы и расширенный алгоритм Евклида

In [6]:
f(x) = 5*x^4 - 15*x^3 + 15*x^2 - 5*x
g(x) = -22*x^5 + 33*x^4 - 11*x^3 - 99*x + 99

show(LatexExpr('Дано:'))
show(LatexExpr(r'f(x) = ' + latex(f(x))))
show(LatexExpr(r'g(x) = ' + latex(g(x))))

Дано:

f(x) = 5 \, x^{4} - 15 \, x^{3} + 15 \, x^{2} - 5 \, x

g(x) = -22 \, x^{5} + 33 \, x^{4} - 11 \, x^{3} - 99 \, x + 99

In [7]:
# Создаем кольцо многочленов с переменными x, y
R2.<x, y> = PolynomialRing(QQ)
f = 5*x^4 - 15*x^3 + 15*x^2 - 5*x
g = -22*x^5 + 33*x^4 - 11*x^3 - 99*x + 99

In [8]:
# НОД при помощи стандартной функции gcd
show("НОД = ", gcd(f, g))

'НОД = ' x - 1

In [9]:
# Вычисление коэффициентов тождества Безу и gcd(a,b)
def bezout(a, b):
    x = 1
    xx = 0
    y = 0
    yy = 1
    while b:
        entire, remainder = a // b, a % b
        a = b
        b = remainder
        x, xx = xx, x - xx * entire
        y, yy = yy, y - yy * entire
    return (x, y, a)

In [10]:
# результат три элемента: u, v, gcd(a,b)
bez = bezout(f, g)

In [11]:
# левая часть тождества
print("НОД = ", f * bez[0] + g * bez[1])

НОД =  -6336/49*x + 6336/49


In [12]:
# правая часть тождества
print("НОД = ", bez[2])

НОД =  -6336/49*x + 6336/49


In [13]:
#проверка тождества Безу f*u + v*g = gcd(f, g)
if (f*bez[0]+bez[1]*g) - bez[2] == 0:
    print("Получено верное тождество")
else:
    print("Получено неверное тождество")

Получено верное тождество


### Задание 3. Вариант 2. уравнение поверхности второго порядка в простраснтве

In [14]:
xmin, xmax = -10, 10
ymin, ymax = -10, 10
zmin, zmax = -10, 10

In [15]:
var("x y z")
f(x, y, z) = 6*x^2 + 12*x*y + 7*y^2 + 2*x*z + 3*z^2 + 5*x + 5*y + 5*z - 18

show(f(x, y, z) == 0)

6*x^2 + 12*x*y + 7*y^2 + 2*x*z + 3*z^2 + 5*x + 5*y + 5*z - 18 == 0

In [16]:
# график исходной функции
implicit_plot3d(f(x=x, y=y, z=z), (x, xmin, xmax), (y, ymin, ymax), (z, zmin, zmax))

Graphics3d Object

In [17]:
# точность вычислений
accurcity = 10

In [18]:
# запишем коэффициенты уравнения поверхности
a11 = 6
a22 = 7
a33 = 3

a12 = 6
a13 = 1
a23 = 0

a0 = -18
a1 = 5 / 2
a2 = 5 / 2
a3 = 5 / 2

A = matrix([
    [a11, a12, a13],
    [a12, a22, a23],
    [a13, a23, a33]
])

a = vector([a1, a2, a3])

show(LatexExpr("\\text{Matrix views}"))
show(LatexExpr("(A|a) = " + latex(A) + latex(a) + "^T"))

\text{Matrix views}

(A|a) = \left(\begin{array}{rrr}
6 & 6 & 1 \\
6 & 7 & 0 \\
1 & 0 & 3
\end{array}\right) \left(\frac{5}{2},\,\frac{5}{2},\,\frac{5}{2}\right) ^T

In [19]:
# Характеристический многочлен
var("z")
# identity_matrix - единичная матрица
detA = A - z * identity_matrix(3) 
detA = det(detA).simplify_full()

show(LatexExpr(r'\det(A - z * E) = ' + latex(detA)))

\det(A - z * E) = -z^{3} + 16 \, z^{2} - 44 \, z + 11

In [20]:
# Собственные числа
roots = solve(detA == 0, z)
eigens = []

for i, root in enumerate(roots):
    if (root.rhs().imag().n(digits = accurcity) < 10 ** -accurcity):
        eigens.append(root.rhs().real().n(digits = accurcity))
        show(LatexExpr("z_{" + str(i) + "} = " + str(eigens[-1])))

z_{0} = 3.153543616

z_{1} = 0.2775206719

z_{2} = 12.56893571

In [21]:
# Собственные вектора
eigenvectors = []
var("x1 x2 x3")

for i, eigen in enumerate(eigens):
    dotA = A - eigen * identity_matrix(3)    
    equations = []
    for j in range(3):
        equations.append(dotA[j][0] * x1 + dotA[j][1] * x2 + dotA[j][2] * x3 == 0)

    equations[1] = equations[1] - equations[0] * dotA[1][0] / dotA[0][0]
    equations[2] = equations[2] - equations[0] * dotA[2][0] / dotA[0][0]
    equations[2] = equations[2] - equations[1] * (equations[2].lhs() / equations[1].lhs())
    
    if (equations[2].lhs() == 0 and equations[2].rhs() == 0):
        equations[2] = (x3 == 1)
    else:
        print("Ранг матрицы равен трём")
    eigenvec = vector([j.rhs().n(digits = accurcity) for j in solve(equations, x1, x2, x3)[0]])
    ans = dotA * eigenvec
    show(LatexExpr(r'\lambda_{' + str(i) + '} = ' + str(eigen.n(digits = accurcity)) + ', s_{' + str(i) + '} = ' + str(eigenvec.n(digits = accurcity))))
    eigenvectors.append(eigenvec.n(digits = accurcity))
    

\lambda_{0} = 3.153543616, s_{0} = (0.1535436158, -0.2395092009, 1.000000000)

\lambda_{1} = 0.2775206719, s_{1} = (-2.722479327, 2.429888612, 1.000000000)

\lambda_{2} = 12.56893571, s_{2} = (9.568935705, 10.30962058, 1.000000000)

In [22]:
# Матрица перехода
S = matrix(eigenvectors)
show(LatexExpr("S = " + latex(S.n(digits = accurcity))))
# Нормируем собсветнные векторы
for eigenvector in eigenvectors:
    eigenvector = eigenvector / sqrt(eigenvector.dot_product(eigenvector))
normirS = matrix(eigenvectors)
# После нормировки получим
show(LatexExpr("S^{*} = " + latex(normirS.n(digits = accurcity))))

S = \left(\begin{array}{rrr}
0.1535436158 & -0.2395092009 & 1.000000000 \\
-2.722479327 & 2.429888612 & 1.000000000 \\
9.568935705 & 10.30962058 & 1.000000000
\end{array}\right)

S^{*} = \left(\begin{array}{rrr}
0.1535436158 & -0.2395092009 & 1.000000000 \\
-2.722479327 & 2.429888612 & 1.000000000 \\
9.568935705 & 10.30962058 & 1.000000000
\end{array}\right)

In [23]:
diagA = normirS * A * normirS.T
show(LatexExpr("S^{*} * A * {S^{*}}^T = " + latex(diagA.n(digits = accurcity))))

S^{*} * A * {S^{*}}^T = \left(\begin{array}{rrr}
3.408792380 & 1.164153218 \times 10^{-9} & 2.444721758 \times 10^{-9} \\
1.164153218 \times 10^{-9} & 3.973055972 & -7.275957614 \times 10^{-10} \\
2.561137080 \times 10^{-9} & -1.862645149 \times 10^{-9} & 2499.368148
\end{array}\right)

In [24]:
# Умножим матрицу на вектор
a1 = S.T * a
show(a1)

(17.49999999, 31.24999998, 7.500000000)

In [25]:
v(x, y, z) = 0
variables = [x, y, z]
for i in range(len(variables)):
    v += diagA[i][i] * variables[i] ^ 2 + variables[i] * a1[i]
show(v(x, y, z) == 0)

3.408792380*x^2 + 3.973055972*y^2 + 2499.368148*z^2 + 17.49999999*x + 31.24999998*y + 7.500000000*z == 0

In [27]:
# Сделаем замену переменных
coef = []

for i in range(len(a1)):
    coef.append((a1[i] / (2 * diagA[i][i])) ** 2)
    if (diagA[i][i] * a1[i] < 0):
        coef[-1] = -coef[-1]

v(x, y, z) = a0
variables = [x, y, z]
for i in range(len(variables)):
    v += diagA[i][i] * (variables[i] + coef[i]) ^ 2 - diagA[i][i] * abs(coef[i])
# После замены получим
show(v(x, y, z) == 0)

3.408792380*(x + 6.588931683)^2 + 3.973055972*(y + 15.46645152)^2 + 2499.368148*(z + 2.251137765e-6)^2 - 101.9150041 == 0

In [28]:
implicit_plot3d(f, (xmin, xmax), (ymin, ymax), (zmin, zmax), color="red")

Graphics3d Object